# Summary
This notebook served as our exploration for the sessions of Congress where we had bill data and were able to extract what subjects Congress was voting on during that session from our data scraping and cleaning phase of the project.

This notebook accomplishes two important things. The first is determing what subjects are most important to each session of Congress. We initially tried attempts like counting the subjects that appeared the most often per session or looking at the intersection of the sets of subjects for all sessions of Congress or even grouping similar sessions together. What we found, ultimately, is that Congress self-references both itself and the federal government as a whole, with the most common subject amongst most sessions being 'Government operations and politics', which isn't particularly insightful.

As a result, we used tfidf to find the most relevant subjects to each session of Congress. We treat each session as essentially a document where a record of a vote on a subject means that subject appears in the document once and 20 votes on a subject means that subject appears 20 times in that document, the corpus being all of our sessions of Congress. Using tfidf, we found subjects relevant to each session of Congress but we were still finding some self-referential subjects, like 'Bill consideration (Rule)' which appeared only in the first few sessions we had this bill data for and thus did not have a significant idf component to outweigh it's high tf score. Our only path was raw filtering of the subjects, filtering out all subjects that contain 'Congress', 'Government', 'Bill', 'Advisory', 'Executive', 'Public', and 'Federal' keywords.

After running tfidf, we knew what subjects were most important to each individual session but we wanted to have more granularity. What we did next was go through all the subjects and record every date where at least one bill concerning one of these most relevant subjects was voted on in Congress. This let us create a very fine resolution graph of the day-to-day of Congress and is contained in our website [here](http://zoherghadyali.me/DataScienceFinalProject/).

Imports the packages we need.

In [1]:
import numpy as np
import pandas as pd
import json

We go through and we look for all sessions of Congress that we know we have subject data for from our previous data scraping and cleaning phase. If any of the .csv files have any sponsors, then that means we have bill data which means we have subjects.

In [6]:
with_subjects = []

for congress_no in range(1, 114):
    for body in ['house', 'senate']:
        df = pd.read_csv('data/congress_sessions_legislation/' + str(congress_no) + body + '.csv', low_memory=False)
        if not df.sponsor[df.sponsor != 'unknown'].empty: #df.sponsor is unknown when we have no bill data, any non "unknown"s means we have bill data
            with_subjects.append(str(congress_no) + body)

print with_subjects

['93house', '93senate', '94house', '94senate', '95house', '95senate', '96house', '97house', '98house', '101house', '101senate', '102house', '102senate', '103house', '103senate', '104house', '104senate', '105house', '105senate', '106house', '106senate', '107house', '107senate', '108house', '108senate', '109house', '109senate', '110house', '110senate', '111house', '111senate', '112house', '112senate', '113house', '113senate']


Now that we know which sessions have subject data, we go through all of them and grab all of the subjects and store each session's subjects as a list.

In [8]:
subjects = []
for session in with_subjects:
    df = pd.read_csv('data/congress_sessions_legislation/' + session + '.csv', low_memory=False)
    columns = list(df.columns.values) #grabs all of the fields in the .csv files, between title and billTitle are all of our subjects
    subjects.append(columns[columns.index('title')+1:columns.index('billTitle')])

Now we apply the filtering on the subjects discussed in the summary at the top of this notebook, filtering out all subjects that refer to 'Congress', 'Government', 'Bill', 'Advisory', 'Executive', 'Federal', and 'Public'.

In [9]:
filteredSubjects = []
for subjectlist in subjects:
    res = []
    for subject in subjectlist:
        if 'Congress' not in subject and 'Government' not in subject and 'Bill' not in subject and 'Advisory' not in subject and 'Executive' not in subject and 'Public' not in subject and 'Federal' not in subject:
            res.append(subject)
    filteredSubjects.append(res)

subjects = filteredSubjects

Now we do the tf part of tfidf where we count, within each document, the subjects that appear the most often. Before filtering, this subject would have been 'Government operations and politics' for most of the sessions.

In [10]:
subjects_count = []
for i in range(0, len(with_subjects)):
    df = pd.read_csv('data/congress_sessions_legislation/' + with_subjects[i] + '.csv', low_memory=False)
    res = []
    for subject in subjects[i]:
        res.append(len(df[subject][df[subject] == 1])) #counts the number of votes that related to this subject (thus the number of times this subject will appear in the document)
    subjects_count.append(res)  

This is just a sanity check to make sure that for every subject we have recorded, we also have a count of how many times bills related to that subject were voted on and to ensure that for each subject, an accurate count was obtained.

In [11]:
print len(subjects[0])
print len(subjects_count[0])  

index = 2
print subjects[0][index]
print subjects_count[0][index]

df = pd.read_csv('data/congress_sessions_legislation/93house.csv', low_memory=False)
print len(df[subjects[0][index]][df[subjects[0][index]] ==1 ])

465
465
Aerial bombing
6
6


The purpose of the subjects_tot_count dictionary is to create a master list of all the subjects in all sessions of Congress which we need for the idf part.

In [21]:
subjects_tot_count = {}
for i in range(0, len(subjects)):
    for j in range(0, len(subjects[i])):
        if subjects[i][j] in subjects_tot_count:
            subjects_tot_count[subjects[i][j]] += subjects_count[i][j]
        else:
            subjects_tot_count[subjects[i][j]] = subjects_count[i][j]

Now we go through all the subjects and count how many sessions, of the 35 we have this bill data for, a given subject appears. We then take the log of the total number of sessions divided by the count to get our idf score.

In [22]:
subjects_idf = {}
for key in subjects_tot_count:
    count = 0
    for subjectlist in subjects:
        if key in subjectlist:
            count += 1
    subjects_idf[key] = np.log(float(len(with_subjects))/count)

We calculate the tfidf score, found by taking the product of the total number of times a subject appears within a given session and the idf score for the given subject, for each subject in each session of Congress. We loop through all of the subjects and find the highest tfidf scorer and thus the most relevant subject to that session (the one that was voted on the most times but is not common to the other sessions).

In [14]:
tfidf = []
for i in range(0, len(subjects)):
    res = []
    for j in range(0, len(subjects[i])):
        res.append(subjects_count[i][j] * subjects_idf[subjects[i][j]])
    tfidf.append(res)

tfidf_subjects = []
for i in range(0, len(with_subjects)):
    print '------------'
    print with_subjects[i]
    print subjects[i][tfidf[i].index(max(tfidf[i]))]
    print max(tfidf[i])
    tfidf_subjects.append(subjects[i][tfidf[i].index(max(tfidf[i]))])
        

------------
93house
Medical care, personnel, and facilities
78.0859332133
------------
93senate
Petroleum and petroleum products
99.1430143432
------------
94house
Petroleum and petroleum products
148.035459773
------------
94senate
Value-added tax
223.975751217
------------
95house
Agriculture and Rural Affairs
270.481510719
------------
95senate
Agriculture and Rural Affairs
215.996026545
------------
96house
Foreign Trade and Investments
319.375650467
------------
97house
Defense articles
154.774353688
------------
98house
Agriculture and Rural Affairs
251.022409228
------------
101house
Narcotic traffic
109.934256147
------------
101senate
Narcotic traffic
149.550204308
------------
102house
Narcotic traffic
206.993329142
------------
102senate
Narcotic traffic
88.1454846585
------------
103house
Narcotic traffic
306.033199545
------------
103senate
Narcotic traffic
168.367779685
------------
104house
Educational policy
412.005860876
------------
104senate
Welfare reform
277.61114

In order to plot our raw findings, which would be the number of bills per year as we did [here](https://github.com/dinopants174/DataScienceFinalProject/blob/gh-pages/images/subject_filtering.png), we need to convert from individual sessions and houses to years. We did this by calculating the year based on the session number and creating a dictionary where the keys are years and the values are lists of the sessions, sometimes both House and Senate, and sometimes only one. We use this to make finding our .csv files and getting our data easier.

We also see several repeats among our tfidf subjects, like 'Narcotic traffic' which has the highest tfidf score for 6 sessions straight so we eliminate the duplicates from our tfidf subject list.

In [23]:
years = [] #years will hold the calculate year for each session, the calculated year for 93house and 93senate will be the same (1973)
           #so we can use that to when we create year_to_session

for session in with_subjects:
    tmp = ''
    for char in session:
        if char.isdigit():
            tmp += char
    if int(tmp) == 93:
        years.append(int(tmp) + 1880) #corresponds to 1973, the first year we have subject data for
    else:
        years.append((int(tmp)-93)*2 + 1880 + 93) #since each session of Congress is 2 years, we have to increment the year by 2 for each session increment

    
year_to_session = {}
for i in range(0, len(years)):
    if years[i] in year_to_session:
        year_to_session[years[i]].append(with_subjects[i])
    else:
        year_to_session[years[i]] = [with_subjects[i]]
        
filtered_tfidf_subjects = list(set(tfidf_subjects))

print filtered_tfidf_subjects
print year_to_session

['Electronic government information', 'Hurricane aftermath legislation', 'Education programs funding', 'Value-added tax', 'Agriculture and Rural Affairs', 'Hazardous waste site remediation', 'Education of disabled students', 'Defense articles', 'Foreign Trade and Investments', 'Research administration and funding', 'Fraud offenses and financial crimes', 'Narcotic traffic', 'Iraq compilation', 'EBB Terrorism', 'Medical care, personnel, and facilities', 'Rural conditions and development', 'Educational policy', 'Petroleum and petroleum products', 'Computers and government', 'Welfare reform']
{2011: ['112house', '112senate'], 1989: ['101house', '101senate'], 1991: ['102house', '102senate'], 1993: ['103house', '103senate'], 2013: ['113house', '113senate'], 1995: ['104house', '104senate'], 1997: ['105house', '105senate'], 1999: ['106house', '106senate'], 2001: ['107house', '107senate'], 2009: ['111house', '111senate'], 2003: ['108house', '108senate'], 1973: ['93house', '93senate'], 2007: ['1

Since the final iteration of the subjects visualization was done in d3.js, we write out the data we gather to an array in Javascript which we can then use to create our d3 visualization.

We start by setting up our load_subjects.js file and then we loop through each of the years and each of the sessions. For each session, we loop through the tfidf subjects and find a dataframe that contains only votes on bills that addressed one of those subjects. We then assemble our JSON object using the data of that vote and the number of bills for each of the tfidf subjects.

This method may produce repeated data, such as if a vote on a bill that concerned 'Electronic government information' also contained 'Educational policy', which means this is inefficient but overall should not affect our visualization other than data being replotted over itself.

In [20]:
asc_years = sorted(year_to_session.keys()) #just means we go in order so our data array is reasonably sorted by year

test = open('load_subjects.js', 'w')

test.write('var data = ')

lst = []

for year in asc_years:
    for session in year_to_session[year]:
        df = pd.read_csv('data/congress_sessions_legislation/' + session + '.csv', low_memory=False)
        for subject in filtered_tfidf_subjects:
            if subject in list(df.columns.values):
                tmp = df[df[subject]==1]
                for date in list(set(tmp.date)):
                    res = {}
                    res['date'] = date
                    for subject2 in filtered_tfidf_subjects:
                        if subject2 in list(tmp.columns.values):
                            res[subject2] = len(tmp[(tmp.date== date) & (tmp[subject2] == 1)])
                        else:
                            res[subject2] = 0
                    lst.append(res)

json.dump(lst, test)